In [1]:
# 1: Загрузка текста статьи
!curl -s "https://en.wikipedia.org/w/index.php?title=Hummingbird&action=raw" > data/article.txt
print("Статья загружена.")
!head -n 5 article.txt

Статья загружена.
head: article.txt: No such file or directory


In [2]:
# 2: Чтение содержимого файла
with open("data/article.txt", "r") as f:
    text = f.read()
print("Содержимое файла прочитано.")
print(text[:500])  # Вывод первых 500 символов для проверки

Содержимое файла прочитано.
{{Short description|Family of birds}}
{{Other uses}}
{{good article}}
{{Use dmy dates|date=March 2023}}
{{Automatic taxobox
|fossil_range = [[Rupelian]] {{Fossilrange|30|0}}
|image = Trinidad and Tobago hummingbirds composite.jpg
|image_caption = Four hummingbirds<br/>from [[Trinidad and Tobago]]
|taxon = Trochilidae
|authority = [[Nicholas Aylward Vigors|Vigors]], 1825
|subdivision_ranks = [[Subfamilies]]
|subdivision = {{collapsible list |title = <small>Subfamiles:</small>|bullets = yes|†''[[E


In [3]:
# 3: Очистка текста
import re


def remove_nested_braces(text):
    """Удаляет все тексты внутри самых внешних фигурных скобок итеративно, пока вложенные скобки не исчезнут."""
    depth = 0
    start = 0
    to_remove = []
    # Перебор текста для определения самых внешних скобок
    for i, char in enumerate(text):
        if char == "{":
            if depth == 0:
                start = i
            depth += 1
        elif char == "}":
            depth -= 1
            if depth == 0:
                to_remove.append(text[start : i + 1])

    # Удаление выделенных сегментов из текста
    for segment in to_remove:
        text = text.replace(segment, "")

    return text


# Обработка текста для удаления вложенных скобок
cleaned_text = remove_nested_braces(text)

# Дополнительная очистка для удаления специфических шаблонов
patterns = [
    (r"\[\[(File|Category):[^\]]+\]\]", ""),  # Удаление ссылок на файлы или категории
    (
        r"\[\[[^\]]+\|([^\]]+)\]\]",
        r"\1",
    ),  # Удаление мета-текста в вики-ссылках, оставляя только полезный текст
    (r"\[\[([^\]]+)\]\]", r"\1"),  # Удаление двойных квадратных скобок вокруг текста
    (r"<[^>]+>", ""),  # Удаление всех HTML-тегов
    (r"<ref.*?\/ref>", "", re.DOTALL),  # Удаление содержимого между тегами <ref>
    (r"<ref.*?\/>", ""),  # Удаление пустых тегов <ref/>
    (r"'''(.*?)'''", r"\1"),  # Удаление тройных кавычек вокруг текста
    (
        r"== See also ==.*",
        "",
        re.DOTALL,
    ),  # Удаление раздела "См. также" и всего, что идет после него
    (r"\]\]", ""),  # Удаление лишних закрывающих квадратных скобок
    (r"\n{2,}", "\n"),  # Уменьшение множественных переводов строк до одного
]

# Применение каждого шаблона очистки
for pattern, repl, *args in patterns:
    cleaned_text = re.sub(pattern, repl, cleaned_text, *args)

print("Очищенный текст:")
print(cleaned_text)

Очищенный текст:

Hummingbirds are birds native to the Americas and comprise the biological family Trochilidae. With approximately 366 species and 113 genera, they occur from Alaska to Tierra del Fuego, but most species are found in Central and South America. As of 2024, 21 hummingbird species are listed as endangered or critically endangered, with numerous species declining in population.
Hummingbirds have varied specialized characteristics to enable rapid, maneuverable flight: exceptional metabolic capacity, adaptations to high altitude, sensitive visual and communication abilities, and long-distance migration in some species. Among all birds, male hummingbirds have the widest diversity of plumage color, particularly in blues, greens, and purples. Hummingbirds are the smallest mature birds, measuring  in length. The smallest is the  bee hummingbird, which weighs less than , and the largest is the  giant hummingbird, weighing . Noted for long beaks, hummingbirds are specialized for fe

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# Настройка конфигурации Spark для использования всех доступных ядер
conf = SparkConf().setAppName("HummingbirdArticleWordCount").setMaster("local[*]")

# Инициализация SparkContext и SparkSession
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

try:
    # Создание RDD из очищенного текста и подсчет частоты слов
    words_rdd = sc.parallelize(cleaned_text.lower().split())
    word_counts = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

    # Преобразование RDD в DataFrame
    words_df = spark.createDataFrame(word_counts, ["Word", "Count"])

    # Сортировка слов по частоте и выбор топ-30
    top_words_df = words_df.orderBy("Count", ascending=False).limit(30)

    # Сохранение результатов в один CSV-файл
    top_words_df.coalesce(1).write.option("header", "true").csv(
        "data/word_count.csv", mode="overwrite"
    )

    print("Результаты сохранены в одном CSV файле 'word_count.csv'.")
except Exception as e:
    print("Произошла ошибка при работе с Spark:", e)
finally:
    # Закрытие SparkContext
    sc.stop()

24/05/04 23:45:14 WARN Utils: Your hostname, MacBook-Pro-Andrej.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
24/05/04 23:45:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/04 23:45:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Результаты сохранены в одном CSV файле 'word_count.csv'.
